**本项目实现了32 * 32矩阵乘法IP核的加速**
==============================

In [107]:
print("This project is designed for speed up Matrix Multiply")

This project is designed for speed up Matrix Multiply


### 加载Overlay
- 我们通过Overlay()方法将生成的硬件设计加载到PL上，
- 通过overlay.axi_dma_0和overlay.axi_dma_1语句，我们可以通过访问的Python对象的形式来与IP交互

In [112]:
import pynq.lib.dma
import numpy as np
overlay = pynq.Overlay("./matrixmul.bit")

dma0 = overlay.axi_dma_0
dma1 = overlay.axi_dma_1

### 分配内存供IP使用
- 在PL中的IP访问DRAM之前，必须为其保留一些内存供IP使用，分配大小与地址
- 我们分别为输入矩阵A、输出矩阵B和结果矩阵Res三个部分分配内存，数据类型为int


In [113]:
from pynq import allocate
rows = cols = 32
A = allocate(shape=(rows,cols), dtype=int)
B = allocate(shape=(rows,cols), dtype=int)
Res = allocate(shape=(rows,cols), dtype=int)

### 读入数据
- 通过np.loadtxt语句，我们可以读入矩阵A和B的数据

In [116]:
# A = np.loadtxt('./input.txt', dtype=int, delimiter=' ')
# B = np.loadtxt('./input.txt', dtype=int, delimiter=' ')
# print(A)
# print(B)

f = open('./input.txt')
lines = f.readlines()
_row = 0
for line in lines:
    list = line.strip('\n').split(' ')
    A[_row:] = list[0:32]
    B[_row:] = list[0:32]
    _row += 1
np.set_printoptions(threshold=np.inf)



### 向ip核传入数据
- 通过dma.sendchannel.transfer和dma.sendchannel.transfer方法可以将数据从输入缓冲区通过管道推送到输出缓冲区

In [117]:
dma0.sendchannel.transfer(A)
dma1.sendchannel.transfer(B)
dma0.recvchannel.transfer(Res)

### 可视化结果
- 最后得到A和B相乘得到的矩阵Res并输出

In [118]:
print("Matrxi A\n")
print(A)
print("\nMatrxi B\n")
print(B)
print("\nMatrxi Res (A*B)\n")
print(Res)

Matrxi A

[[4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]
 [4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]
 [4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]
 [4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]
 [4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]
 [4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]
 [4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]
 [4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 

### 验证结果正确性

In [119]:
Ans = A @ B   #使用python内置矩阵乘法计算
print((Ans == Res).all())

True


**结果为True，说明python内置函数计算结果与ip核计算结果相同，结果正确**